In [1]:
import os
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from lsst.daf.persistence import Butler
import fakeInjectionPipeline as fip
import data_process_utils as dpu

from scipy.stats import norm
import scipy.stats as stats

import imp
imp.reload(fip) 

<module 'fakeInjectionPipeline' from '/global/cscratch1/sd/shl159/projects/dia_improvement/scripts/fakeInjectionPipeline.py'>

In [2]:
!pwd

/global/cscratch1/sd/shl159/projects/dia_improvement/scripts


In [3]:
def get_calexp_injection_dict(patch_list, host_mag_range_list):
    
    conn = sqlite3.connect('../data/table/gal_4639/injection_coord.sqlite')
    calexp_info_dict = {}
    injection_coord_dict = {}

    for patch in patch_list:
        patch_1, patch_2 = patch[0], patch[1]
        


        for host_mag in host_mag_range_list:
            min_mag = host_mag[0]
            max_mag = host_mag[1]

            calexp_info_query = (f'SELECT distinct(visit), filter, detector FROM injection_coord WHERE patch_1={patch_1} AND '
                                 f'patch_2={patch_2} AND min_host_mag = {min_mag} AND max_host_mag = {max_mag}'
                                )
            calexp_info = pd.read_sql_query(calexp_info_query, conn)

            calexp_info_dict[f'{patch}_{min_mag}_{max_mag}'] = calexp_info

        for host_mag in host_mag_range_list:
            min_mag = host_mag[0]
            max_mag = host_mag[1]
            calexp_info = calexp_info_dict[f'{patch}_{min_mag}_{max_mag}']
            for idx, row in calexp_info.iterrows():
                visit = row['visit']
                filt = row['filter']
                detector = row['detector']
                coord_query = (
                    f'SELECT calexp_coord_x, calexp_coord_y FROM injection_coord '
                    f"WHERE patch_1={patch_1} AND patch_2={patch_2} AND min_host_mag = {min_mag} AND max_host_mag = {max_mag} "
                    f"AND visit = {visit} AND filter = '{filt}' AND detector = {detector}"
                )
                coord = pd.read_sql_query(coord_query, conn).to_numpy()
                injection_coord_dict[f'{patch}_{int(min_mag)}_{int(max_mag)}_{int(visit)}_{int(detector)}_{filt}'] = coord
    return calexp_info_dict, injection_coord_dict

In [4]:
!pwd

/global/cscratch1/sd/shl159/projects/dia_improvement/scripts


# injection

In [6]:
patch_list = ['00', '11', '22', '33', '44', '55', '66']
host_mag_list = ['20_21', '21_22', '22_23', '23_24', '24_25']
fake_mag_list = [23.0, 23.1, 23.2, 23.3, 23.4, 23.5, 23.6, 23.7, 23.8, 23.9, 24.0]

host_mag_range_list = [(20, 21), (21, 22), (22,23), (23, 24), (24, 25)]
calexp_info_dict, injection_coord_dict = get_calexp_injection_dict(patch_list, host_mag_range_list)



In [7]:
!pwd

/global/cscratch1/sd/shl159/projects/dia_improvement/scripts


In [7]:
fake_dir = f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/injection/perfect_faint'

injector = fip.fakeInjectionPipeline(
    fake_dir = fake_dir, patch_list = patch_list,
    host_mag_list=host_mag_list, fake_mag_list=fake_mag_list,
    calexp_info_dict=calexp_info_dict, injection_coord_dict=injection_coord_dict
)

In [14]:
# injector.make_fake_repo()
# injector.inject_fakes(poisson=False)

patch:  00
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  11
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  22
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  33
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  44
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  55
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  66
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


injection is done


In [ ]:
fake_dir = f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/injection/perfect_faint'



In [8]:
cf_type = 'default_3sig'

config = f'./config/diffimConfig_al_{cf_type}.py'
file_path = f'../al_faint_perfect.sh'
diff_dir =  f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_faint_{cf_type}'
# injector.get_subtraction_script(diff_dir , config=config, file_path=file_path, use_slurm=True)

# exsistance

In [10]:
for patch in patch_list:
    
    for host_mag in host_mag_list:
        calexp_info = calexp_info_dict[f'{patch}_{host_mag}']
        
        for index, row in calexp_info.iterrows():
            visit = row['visit']
            filt = row['filter']
            detector = row['detector']

            for fake_mag in fake_mag_list:
                fake_mag_str = str(fake_mag).replace('.', '')

                diff_path = os.path.join(
                    diff_dir, f'{patch}_{host_mag}_{visit}_{detector}_{filt}/diff_{fake_mag_str}/deepDiff'
                )
                
                if not os.path.exists(diff_path):
                    print(diff_path)

print()


In [40]:
cf_type = 'default_3sig'
diff_dir =  f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_{cf_type}'
flux_dir = (
    f'/global/cscratch1/sd/shl159/projects/dia_improvement/data'
    f'/patch_0to6/diff/perfect/al_{cf_type}/detection'
)

injector.get_detection(diff_dir=diff_dir, flux_dir=flux_dir, db_name=f'detection.sqlite')
print('done')

patch:  00
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  11
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  22
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  33
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  44
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  55
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  66
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


detection results are saved in /global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/detection/detection.sqlite
fake_src contains the fake information, and artifact contains the artifact information
done


In [16]:
flux_db_path = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/detection/detection.sqlite'
coord_dir = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/coord'
injector.get_detection_coord(flux_db_path, coord_dir)

forced photometry coordinates are saved in /global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/coord


In [26]:
cf_type = 'default_3sig'
diff_dir =  f'/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_{cf_type}'
coord_dir = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/coord'
forced_dir = '/global/cscratch1/sd/shl159/projects/dia_improvement/data/patch_0to6/diff/perfect/al_default_3sig/forced'

config = './config/forcedConfig.py'
file_path = '../forced_al_default_3sig.sh'
injector.get_forced_phot_script(diff_dir, coord_dir, forced_dir, config, file_path=file_path)

patch:  00
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  11
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  22
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  33
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  44
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  55
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


patch:  66
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25


please run this file: ../forced_al_default_3sig.sh


In [7]:
patch = 0
diff_dir = f'/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/{cf_type}'
flux_dir = f'/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/{cf_type}/detection'

injector.get_detection(diff_dir=diff_dir, flux_dir=flux_dir, db_name=f'patch_{patch}{patch}_detection.sqlite')
print('done')

patch:  00
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25
host mag:  25_26
host mag:  26_27
host mag:  27_28


detection results are saved in /global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/default_3sig/detection/patch_00_detection.sqlite
fake_src contains the fake information, and artifact contains the artifact information
done


In [13]:

flux_db_path = '/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/default_3sig/detection/patch_00_detection.sqlite'
coord_dir = '/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/default_3sig/detection/coord_00'

injector.get_detection_coord(flux_db_path, coord_dir)

forced photometry coordinates are saved in /global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/default_3sig/detection/coord_00


In [16]:
coord_dir = '/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/default_3sig/detection/coord_00'


forced_dir = '/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/default_3sig/detection/forced_00'
config = './forcedConfig.py'
file_path = '../forcd_00.sh'
injector.get_forced_phot_script(diff_dir, coord_dir, forced_dir, config, file_path=file_path)

patch:  00
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25
host mag:  25_26
host mag:  26_27
host mag:  27_28


please run this file: ../forcd_00.sh


In [14]:
print('yes')

yes


# patch 1,1

In [20]:
patch = 1
patch_list = [f'{patch}{patch}']


calexp_info_dict_11, injection_coord_dict_11 = get_calexp_injection_dict(patch)

fake_dir = f'/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6'
patch_list = [f'{patch}{patch}']
host_mag_list = ['20_21', '21_22', '22_23', '23_24', '24_25', '25_26', '26_27', '27_28']
fake_mag_list = [20, 20.5, 21, 21.5, 22, 22.5, 23, 23.5, 24, 24.5, 25]

injector_11 = fip.fakeInjectionPipeline(fake_dir = fake_dir,
                                     patch_list = patch_list,
                                     host_mag_list=host_mag_list,
                                     fake_mag_list=fake_mag_list,
                                     calexp_info_dict=calexp_info_dict_11,
                                     injection_coord_dict=injection_coord_dict_11)

In [14]:
# injector_11.make_fake_repo()
# injector_11.inject_fakes()

patch:  11
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25
host mag:  25_26
host mag:  26_27
host mag:  27_28


injection is done


In [21]:
cf_type = 'default_3sig'
patch = 1

config = f'diffimConfig_al_{cf_type}.py'
file_path = f'../subtraction_{cf_type}_{patch}{patch}.sh'
diff_dir = f'/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6/{cf_type}'
injector.get_subtraction_script(diff_dir , config=config, file_path=file_path)

please run this file: ../subtraction_default_3sig_11.sh


In [ ]:
file_path='./subtraction.sh'

In [22]:
!pwd

/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/scripts


# patch 2

In [15]:
# change patch
patch = 2
# rename calexp_info_dict, injection_coord_dict
calexp_info_dict_22, injection_coord_dict_22 = get_calexp_injection_dict(patch)

fake_dir = f'/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6'
patch_list = [f'{patch}{patch}']
host_mag_list = ['20_21', '21_22', '22_23', '23_24', '24_25', '25_26', '26_27', '27_28']
fake_mag_list = [20, 20.5, 21, 21.5, 22, 22.5, 23, 23.5, 24, 24.5, 25]

# rename injector, change calexp_info_dict, injection_coord_dict
injector_22 = fip.fakeInjectionPipeline(fake_dir = fake_dir,
                                     patch_list = patch_list,
                                     host_mag_list=host_mag_list,
                                     fake_mag_list=fake_mag_list,
                                     calexp_info_dict=calexp_info_dict_22,
                                     injection_coord_dict=injection_coord_dict_22)

In [16]:
injector_22.make_fake_repo()
injector_22.inject_fakes()

patch:  22
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25
host mag:  25_26
host mag:  26_27
host mag:  27_28


injection is done


In [17]:
# change patch
patch = 3
# rename calexp_info_dict, injection_coord_dict
calexp_info_dict_33, injection_coord_dict_33 = get_calexp_injection_dict(patch)

fake_dir = f'/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6'
patch_list = [f'{patch}{patch}']
host_mag_list = ['20_21', '21_22', '22_23', '23_24', '24_25', '25_26', '26_27', '27_28']
fake_mag_list = [20, 20.5, 21, 21.5, 22, 22.5, 23, 23.5, 24, 24.5, 25]

# rename injector, change calexp_info_dict, injection_coord_dict
injector_33 = fip.fakeInjectionPipeline(fake_dir = fake_dir,
                                     patch_list = patch_list,
                                     host_mag_list=host_mag_list,
                                     fake_mag_list=fake_mag_list,
                                     calexp_info_dict=calexp_info_dict_33,
                                     injection_coord_dict=injection_coord_dict_33)

In [18]:
injector_33.make_fake_repo()
injector_33.inject_fakes()

patch:  33
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25
host mag:  25_26
host mag:  26_27
host mag:  27_28


injection is done


# patch 4

In [19]:
# change patch
patch = 4
# rename calexp_info_dict, injection_coord_dict
calexp_info_dict_44, injection_coord_dict_44 = get_calexp_injection_dict(patch)

fake_dir = f'/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6'
patch_list = [f'{patch}{patch}']
host_mag_list = ['20_21', '21_22', '22_23', '23_24', '24_25', '25_26', '26_27', '27_28']
fake_mag_list = [20, 20.5, 21, 21.5, 22, 22.5, 23, 23.5, 24, 24.5, 25]

# rename injector, change calexp_info_dict, injection_coord_dict
injector_44 = fip.fakeInjectionPipeline(fake_dir = fake_dir,
                                     patch_list = patch_list,
                                     host_mag_list=host_mag_list,
                                     fake_mag_list=fake_mag_list,
                                     calexp_info_dict=calexp_info_dict_44,
                                     injection_coord_dict=injection_coord_dict_44)

In [20]:
injector_44.make_fake_repo()
injector_44.inject_fakes()

patch:  44
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25
host mag:  25_26
host mag:  26_27
host mag:  27_28


injection is done


# patch 5

In [21]:
# change patch
patch = 5
# rename calexp_info_dict, injection_coord_dict
calexp_info_dict_55, injection_coord_dict_55 = get_calexp_injection_dict(patch)

fake_dir = f'/global/cfs/cdirs/desc-sn/dia/data/shl159/projects/dia_improvement/data/patch_0to6'
patch_list = [f'{patch}{patch}']
host_mag_list = ['20_21', '21_22', '22_23', '23_24', '24_25', '25_26', '26_27', '27_28']
fake_mag_list = [20, 20.5, 21, 21.5, 22, 22.5, 23, 23.5, 24, 24.5, 25]

# rename injector, change calexp_info_dict, injection_coord_dict
injector_55 = fip.fakeInjectionPipeline(fake_dir = fake_dir,
                                     patch_list = patch_list,
                                     host_mag_list=host_mag_list,
                                     fake_mag_list=fake_mag_list,
                                     calexp_info_dict=calexp_info_dict_55,
                                     injection_coord_dict=injection_coord_dict_55)

In [22]:
injector_55.make_fake_repo()
injector_55.inject_fakes()

patch:  55
host mag:  20_21
host mag:  21_22
host mag:  22_23
host mag:  23_24
host mag:  24_25
host mag:  25_26
host mag:  26_27
host mag:  27_28


injection is done


# 

In [ ]:
# 8 host mag bins, 10 visits, 11 fake mag bins, 880 subtractions in total
# it takes 60 hours (3600min) for subtraction
(60 * 60) / (8 * 10 * 11 )

4.090909090909091